In [1]:
import sys
sys.path.append("../")
import io
from glob import glob
from tqdm.contrib.concurrent import process_map
from document_search.ocr import DocumentReader, EntityProcessor


/home/atom/anaconda3/envs/doc-search2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Adding all documents to storage

In [2]:
documents = glob('../data_all/*.pdf')
len(documents)


17

In [3]:
def read_document(document_path):
    doc_reader = DocumentReader()
    with open(document_path, 'rb') as f:
        raw_document = io.BytesIO(f.read())
    processed_doc, errors = doc_reader.read(document_path)
    print(document_path, errors)
    processed_doc.entities = EntityProcessor.merge_text_entities(processed_doc.entities)
    processed_doc.entities = EntityProcessor.filter_short_entities(processed_doc.entities)
    return processed_doc, raw_document


doc_reader = DocumentReader()

raw_documents = []
processed_docs = []
for processed_doc, raw_document in process_map(read_document, documents, max_workers=4):
#for processed_doc, raw_document in map(read_document, documents): 
    raw_documents.append(raw_document)
    processed_docs.append(processed_doc)


  0%|          | 0/17 [00:00<?, ?it/s]

../data_all/СП_54_13330_2016_«СНиП_31_01_2003_Здания_жилые_многоквартирные».pdf []


  6%|▌         | 1/17 [00:04<01:12,  4.53s/it]

../data_all/СП 52.13330.2016.pdf []
../data_all/СанПиН 2.1.2.2645-10.pdf []
../data_all/СНиП 12-01-2004 (СП 48.13330.2019).pdf []


 12%|█▏        | 2/17 [00:18<02:31, 10.12s/it]

...

 59%|█████▉    | 10/17 [01:05<00:47,  6.83s/it]

../data_all/SP.13130.2023.pdf []
../data_all/СП 20.13330.2016.pdf []


 88%|████████▊ | 15/17 [01:29<00:11,  5.82s/it]

../data_all/СП 50.13330.2012.pdf []


100%|██████████| 17/17 [01:31<00:00,  5.39s/it]


In [4]:
from document_search.storages import DocumentStorageE5
from document_search.search import TextEntityEmbedderE5

embedder = TextEntityEmbedderE5()
storage = DocumentStorageE5(embedder)


/home/atom/igor/hacks/document-search/notebooks/../document_search/search/embedders.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  self.embedder = HuggingFaceEmbeddings(


In [5]:
for doc, raw_doc in zip(processed_docs, raw_documents):
    storage.add_document(doc, raw_doc, pbar=True)


0it [00:00, ?it/s]
73it [00:57,  1.28it/s]                        
0it [00:00, ?it/s]
100%|██████████| 20/20 [00:16<00:00,  1.22it/s]
55it [00:52,  1.05it/s]                        
120it [00:50,  2.38it/s]                         
81it [00:29,  2.72it/s]                        
5it [00:04,  1.05it/s]                       
0it [00:00, ?it/s]
194it [04:36,  1.43s/it]                         
90it [00:49,  1.80it/s]                        
91it [01:10,  1.30it/s]                        
37it [00:15,  2.38it/s]                        
23it [00:08,  2.84it/s]                        
107it [00:41,  2.56it/s]                         
119it [00:59,  1.99it/s]                         
63it [00:44,  1.42it/s]                        


## Test search

In [6]:
from document_search.rag.retrievers import YandexGPTRetriever

rag_retriever = YandexGPTRetriever(embedder=embedder, storage=storage)


/home/atom/igor/hacks/document-search/notebooks/../document_search/rag/retrievers.py:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  self.llm_chain = LLMChain(llm=self.llm, prompt=self.prompt)


In [7]:
query = "какие трубопроводы не допускается прокладывать через помещения вентиляционного оборудования?"

data, answer = rag_retriever.retrieve_answer_detailed(query, k=5, rag_k=1, context_length=0)
print(answer)


/home/atom/igor/hacks/document-search/notebooks/../document_search/rag/retrievers.py:59: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  answer = self.llm_chain.run(


**Ответ на вопрос пользователя:**

**Какие трубопроводы не допускается прокладывать через помещения вентиляционного оборудования?**

В соответствии с пунктом 10.6 документа, через помещения для вентиляционного оборудования не допускается прокладывать следующие трубопроводы:

а) с легковоспламеняющимися и горючими жидкостями и газами;

б) канализационные с прочистками и ревизиями (кроме трубопроводов ливневой канализации и водоотведения из вышележащих помещений для вентиляционного оборудования, в том числе от вентиляционного оборудования).

**Обоснование ответа:**

Данный запрет обусловлен необходимостью обеспечения безопасности и предотвращения возможных аварий и чрезвычайных ситуаций, связанных с использованием легковоспламеняющихся и горючих материалов, а также для предотвращения засоров и обеспечения нормального функционирования системы водоотведения.

Следует отметить, что данный ответ является предварительным и основан на информации, представленной в указанных отрывках документов.

In [8]:
data


[(TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=66, document_id='63d55c76804640a98d1ebbcf281b68a5'), text='10.6  Через  помещение  для  вентиляционного  оборудования  не  допускается  прокладывать трубопроводы:'),
  0.91254383),
 (TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=77, document_id='63d55c76804640a98d1ebbcf281b68a5'), text='Не  допускается  прокладка  магистральных  и  разводящих  трубопроводов  систем  отопления  и внутреннего теплоснабжения через помещения жилых квартир, а также установка в них арматуры и спускных устройств общедомовых систем.'),
  0.8962804),
 (TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=50, document_id='63d55c76804640a98d1ebbcf281b68a5'), text='7.10.14  Оборудование  вытяжных  систем  производственных,  административно-бытовых  и общественных  зданий,  удаляющих  воздух  с  резким  или  неприятным  запахом  (из  общественных убор

## Run tests

In [12]:
import pandas as pd

test_data = pd.read_csv('test_data.csv')
for test_case in test_data.to_dict('records'):
    query = test_case['question']
    data, answer = rag_retriever.retrieve_answer_detailed(query, k=5, rag_k=1, context_length=0)
    print('-'*40)
    print('QUERY:')
    print(query)
    print('ANSWER:')
    print(answer)
    print('SEARCH DATA:')
    for res in data:
        print(res)
    print("GROUND TRUTH:")
    print(test_case['ground_truth'])


----------------------------------------
QUERY:
Необходимо ли отражать перечень помещений и оборудования, подлежащих защите автоматическими установками пожаротушения при разработке проектной документации объекта капитального строительства согласно 87 постановлению?
ANSWER:
**Вопрос пользователя:** Необходимо ли отражать перечень помещений и оборудования, подлежащих защите автоматическими установками пожаротушения при разработке проектной документации объекта капитального строительства согласно 87 постановлению?

**Ответ:**

В соответствии с пунктом «з» раздела 9 «Мероприятия по обеспечению пожарной безопасности» проектной документации объекта капитального строительства необходимо отразить перечень зданий, сооружений, помещений и оборудования, которые подлежат защите автоматическими установками пожаротушения.

Таким образом, при разработке проектной документации необходимо включить в неё перечень помещений и оборудования, требующих установки автоматических установок пожаротушения, с цел

Retrying langchain_community.llms.yandex.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised _MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.RESOURCE_EXHAUSTED
	details = "ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {created_time:"2024-09-10T22:12:15.676572868+00:00", grpc_status:8, grpc_message:"ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests"}"
>.


----------------------------------------
QUERY:
При разработке раздела система газоснабжения, ннеобходимо ли описывать способы контроля температуры - для объектов производственного назначения?
ANSWER:
**Ответ на вопрос пользователя**:

На основе предоставленных отрывков из документов можно сделать вывод, что для объектов производственного назначения в системе газоснабжения **необходимо описывать способы контроля температуры**. Это требование содержится в пункте и) подраздела «Система газоснабжения» раздела 5.

Однако способ контроля температуры и его технические решения не указаны конкретно. Вероятно, они зависят от типа оборудования, используемого на производстве, и от технических условий источника газоснабжения. Для получения более подробной информации о способах контроля температуры следует обратиться к техническим условиям и проектной документации.
SEARCH DATA:
(TextDocEntity(position=EntityPosition(document_name='Postanovlenie-Pravitelstva-RF-_-87_red-ot-27.05.2022_.pdf', page_num

Retrying langchain_community.llms.yandex.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised _MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.RESOURCE_EXHAUSTED
	details = "ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {created_time:"2024-09-10T22:12:24.940049708+00:00", grpc_status:8, grpc_message:"ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests"}"
>.


----------------------------------------
QUERY:
Какой коэффициент естественной освещенности должен быть на кухне?
ANSWER:
**Ответ**:

Коэффициент естественной освещённости (КЕО) в жилых комнатах и кухнях должен быть не менее 0,5 %.

В контексте СанПиН 2.1.2.2645—10, пункт 5.2, указано, что коэффициент естественной освещённости в жилых комнатах и кухнях жилых домов должен быть не менее 0,5 % через светопроёмы в наружных ограждающих конструкциях здания при одностороннем боковом освещении.

Для кухни требование к КЕО составляет не менее 0,5 %, что является обязательным для обеспечения достаточного естественного освещения в помещении.
SEARCH DATA:
(TextDocEntity(position=EntityPosition(document_name='СанПиН 2.1.2.2645-10.pdf', page_number=13, document_id='cf8ef568ba014671a275b8919d8c78df'), text='5.2.  Коэффициент  естественной  освещенности  (далее -  КЕО)  в  жилых комнатах и кухнях должен быть не менее 0,5 %.'), 0.88974947)
(TextDocEntity(position=EntityPosition(document_name='СанПиН 2.

Retrying langchain_community.llms.yandex.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised _MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.RESOURCE_EXHAUSTED
	details = "ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {grpc_message:"ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests", grpc_status:8, created_time:"2024-09-10T22:12:27.743854246+00:00"}"
>.


----------------------------------------
QUERY:
Можно ли разместить ванную комнату над кухней?
ANSWER:
**Ответ:**

Вопрос о размещении ванной комнаты над кухней регулируется несколькими пунктами из СанПиН 2.1.2.2645— 10 и других указанных документов.

В документе упоминается, что размещение ванной комнаты над кухней допускается только в двухуровневых квартирах. Однако прямого разрешения на размещение ванной комнаты над кухней в обычных квартирах не даётся.

Также в документе перечисляются требования к размещению помещений общественного назначения в жилых зданиях. Помещения общественного назначения, в том числе ванные комнаты, должны иметь изолированные входы, не связанные с жилой частью здания. Это может ограничивать возможность размещения ванной комнаты над кухней.

Относительно размещения гаражей под жилыми зданиями существуют ограничения: запрещено размещать над гаражами помещения для работы с детьми и лечебно-профилактического назначения. Однако про размещение ванных комнат над гар

Retrying langchain_community.llms.yandex.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised _MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.RESOURCE_EXHAUSTED
	details = "ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {grpc_message:"ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests", grpc_status:8, created_time:"2024-09-10T22:12:40.709950903+00:00"}"
>.


----------------------------------------
QUERY:
Какая должна быть ширина лестницы для жилых зданий коридорного типа?
ANSWER:
**Ответ**:

Для пожаробезопасных зон, размещённых не выше 5-го этажа, ширина лестницы 3-го типа должна составлять не менее 1,5 метра.

**Обоснование**:

В рассматриваемом пункте документа говорится, что для пожаробезопасных зон, размещенных не выше 5-го этажа, ширина лестницы 3-го типа должна предусматриваться не менее 1,5 метров. Однако не уточняется, о каком именно типе лестницы идёт речь. Вероятно, имеется в виду лестница, которая ведёт на эвакуационную лестничную клетку. Однако для более точного ответа необходимо уточнение в документе.
SEARCH DATA:
(TextDocEntity(position=EntityPosition(document_name='SP.13130.2023.pdf', page_number=62, document_id='25ebf079eece4befa9d33ce24fd4e91f'), text='на  лестницу  3-го  типа  непосредственно  или  через  коридор.  При  этом  такие  лестницы допускается использовать для пожаробезопасных зон, размещенных не выше 5-го эта